In [1]:
!pip install transformers

In [30]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel

  # Carga el modelo y el procesador.
print("Cargando modelo y procesador CLIP...")
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
print("Modelo y procesador cargados.")

def get_image_embedding(caption: str, image: Image.Image) -> torch.Tensor:
  """
  Genera el embedding para una imagen dada, cargando el modelo CLIP en cada llamada.

  **Advertencia de rendimiento:** Cargar el modelo y el procesador en cada
  llamada es ineficiente. Para procesar múltiples imágenes, es mejor
  cargarlos una sola vez fuera de la función.

  Args:
    image: La imagen para la cual generar el embedding (objeto PIL.Image).

  Returns:
    Un tensor de PyTorch que representa el embedding de la imagen.
  """

  inputs = processor(text=[caption],images=image, return_tensors="pt")
  
  outputs = model(**inputs)
  return outputs.image_embeds

Cargando modelo y procesador CLIP...


config.json: 0.00B [00:00, ?B/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Modelo y procesador cargados.


In [21]:
def calculate_cosine_similarity(embedding1: torch.Tensor, embedding2: torch.Tensor) -> float:
  """
  Calcula la similitud del coseno entre dos embeddings.

  Args:
    embedding1: El primer tensor de embedding.
    embedding2: El segundo tensor de embedding.

  Returns:
    El valor de la similitud del coseno como un flotante.
  """
  cos = torch.nn.CosineSimilarity(dim=1)
  similarity = cos(embedding1, embedding2)
  return similarity.item()


In [27]:
def compare_images_from_urls(url1: str, url2: str):
  """
  Descarga dos imágenes desde URLs, calcula sus embeddings y muestra su similitud.

  Args:
    url1: La URL de la primera imagen.
    url2: La URL de la segunda imagen.
  """
  print(f"Comparando imágenes:\n- URL 1: {url1}\n- URL 2: {url2}\n")
  try:
    # Descargar y abrir las imágenes
    print("Descargando imagen 1...")
    image1 = Image.open(requests.get(url1, stream=True).raw)
    
    print("Descargando imagen 2...")
    image2 = Image.open(requests.get(url2, stream=True).raw)
    print("Imágenes descargadas.\n")

    # Generar embeddings (esto llamará a la función que carga el modelo dos veces)
    print("--- Generando embedding para imagen 1 ---")
    embedding1 = get_image_embedding("",image1)
    
    print("\n--- Generando embedding para imagen 2 ---")
    embedding2 = get_image_embedding("",image2)

    # Calcular similitud
    similarity = calculate_cosine_similarity(embedding1, embedding2)

    # Imprimir resultado
    print(f"\nResultado final:")
    print(f"La similitud del coseno entre las dos imágenes es: {similarity:.4f}")

  except Exception as e:
    print(f"Ha ocurrido un error: {e}")


In [31]:
url1="https://imgs.search.brave.com/hZmuYmoHPsM1Zq6NV0S7epvi4Re79KGLDilDiggk14A/rs:fit:860:0:0:0/g:ce/aHR0cHM6Ly9tLm1l/ZGlhLWFtYXpvbi5j/b20vaW1hZ2VzL0kv/NzFXdUpZYjVSMUwu/anBn"
url2="https://imgs.search.brave.com/D_bWLcxtpywNJRWPjZTqpIQm5x8aP-PfdQaTUYdB3_A/rs:fit:860:0:0:0/g:ce/aHR0cHM6Ly9tLm1l/ZGlhLWFtYXpvbi5j/b20vaW1hZ2VzL0kv/ODF2cWhqUERDRUwu/anBn"
compare_images_from_urls(url1, url2)

Comparando imágenes:
- URL 1: https://imgs.search.brave.com/hZmuYmoHPsM1Zq6NV0S7epvi4Re79KGLDilDiggk14A/rs:fit:860:0:0:0/g:ce/aHR0cHM6Ly9tLm1l/ZGlhLWFtYXpvbi5j/b20vaW1hZ2VzL0kv/NzFXdUpZYjVSMUwu/anBn
- URL 2: https://imgs.search.brave.com/D_bWLcxtpywNJRWPjZTqpIQm5x8aP-PfdQaTUYdB3_A/rs:fit:860:0:0:0/g:ce/aHR0cHM6Ly9tLm1l/ZGlhLWFtYXpvbi5j/b20vaW1hZ2VzL0kv/ODF2cWhqUERDRUwu/anBn

Descargando imagen 1...
Descargando imagen 2...
Imágenes descargadas.

--- Generando embedding para imagen 1 ---

--- Generando embedding para imagen 2 ---

Resultado final:
La similitud del coseno entre las dos imágenes es: 0.6738
